In [1]:
import pandas as pd
import numpy as np
import json
from datetime import datetime, timedelta
from typing import Dict, List, Tuple, Optional
import warnings
import re
from collections import defaultdict
warnings.filterwarnings('ignore')

# COMPLETE SELF-CONTAINED MENTAL HEALTH PREDICTION SYSTEM
# All steps 1-6 in one notebook-ready file

# Step 1: Data Structure Setup
class MentalHealthDataSetup:
    def __init__(self):
        # Keyword dictionaries based on DASS-21 and clinical indicators
        self.depression_keywords = [
            'hopeless', 'worthless', 'empty', 'sad', 'depressed', 'tired', 'exhausted',
            'meaningless', 'unmotivated', 'lonely', 'isolated', 'numb', 'defeated',
            'failure', 'burden', 'pointless', 'dark', 'heavy'
        ]
        
        self.anxiety_keywords = [
            'worried', 'anxious', 'panic', 'nervous', 'scared', 'fear', 'terrified',
            'overwhelmed', 'racing thoughts', 'restless', 'tense', 'on edge',
            'catastrophic', 'doom', 'dread', 'jittery', 'shaky'
        ]
        
        self.stress_keywords = [
            'stressed', 'pressure', 'overwhelmed', 'burned out', 'rushed', 'frantic',
            'overloaded', 'deadline', 'hectic', 'chaotic', 'demanding', 'exhausting',
            'urgent', 'intense', 'suffocating'
        ]
        
        self.suicidal_keywords = [
            'suicide', 'kill myself', 'end it all', 'not worth living', 'better off dead',
            'want to die', 'no point', 'can\'t go on', 'give up', 'escape',
            'disappear', 'harm myself', 'self harm'
        ]
        
        self.positive_keywords = [
            'happy', 'good', 'great', 'excellent', 'amazing', 'wonderful', 'fantastic',
            'peaceful', 'calm', 'relaxed', 'confident', 'optimistic', 'grateful',
            'accomplished', 'proud', 'energetic', 'motivated'
        ]
        
        # Mindfulness habits tracking
        self.mindfulness_habits = [
            'morning_meditation', 'gratitude_journal', 'nature_walk', 'box_breathing',
            'progressive_relaxation', 'mindful_eating', 'digital_detox', 'stretching',
            'journaling', 'cold_shower'
        ]

# Step 2: Feature Engineering
class FeatureEngineer:
    def __init__(self, setup):
        self.setup = setup
    
    def extract_text_features(self, text):
        """Extract features from text input"""
        if not isinstance(text, str):
            text = str(text)
        
        text_lower = text.lower()
        features = {
            'depression_keyword_count': 0,
            'anxiety_keyword_count': 0,
            'stress_keyword_count': 0,
            'suicidal_keyword_count': 0,
            'positive_keyword_count': 0,
            'text_length': len(text),
            'exclamation_count': text.count('!'),
            'question_count': text.count('?')
        }
        
        # Count keywords
        for keyword in self.setup.depression_keywords:
            if keyword in text_lower:
                features['depression_keyword_count'] += 1
                
        for keyword in self.setup.anxiety_keywords:
            if keyword in text_lower:
                features['anxiety_keyword_count'] += 1
                
        for keyword in self.setup.stress_keywords:
            if keyword in text_lower:
                features['stress_keyword_count'] += 1
                
        for keyword in self.setup.suicidal_keywords:
            if keyword in text_lower:
                features['suicidal_keyword_count'] += 1
                
        for keyword in self.setup.positive_keywords:
            if keyword in text_lower:
                features['positive_keyword_count'] += 1
        
        # Calculate sentiment ratio
        total_negative = (features['depression_keyword_count'] + 
                         features['anxiety_keyword_count'] + 
                         features['stress_keyword_count'])
        total_positive = features['positive_keyword_count']
        
        if total_negative + total_positive > 0:
            features['sentiment_ratio'] = total_positive / (total_negative + total_positive)
        else:
            features['sentiment_ratio'] = 0.5  # neutral
            
        return features

# Step 3: Rule-Based Model
class MentalHealthClassifier:
    def __init__(self, setup):
        self.setup = setup
        
        # Define scoring weights for different features
        self.weights = {
            'mood_weight': 0.25,
            'habits_weight': 0.15,
            'text_weight': 0.3,
            'dass_weight': 0.2,
            'trend_weight': 0.1
        }
        
        # Define thresholds for classification
        self.thresholds = {
            'suicidal_ideation': 0.85,
            'severe_depression': 0.75,
            'moderate_depression': 0.6,
            'severe_anxiety': 0.7,
            'moderate_anxiety': 0.55,
            'severe_stress': 0.7,
            'moderate_stress': 0.55,
            'healthy': 0.3
        }
    
    def calculate_risk_scores(self, row):
        """Calculate risk scores for different conditions"""
        scores = {}
        
        # Suicidal ideation score (highest priority)
        suicidal_score = 0
        if row['suicidal_keyword_count'] > 0:
            suicidal_score += 0.8
        if row['mood_score'] <= 2:
            suicidal_score += 0.6
        if row.get('avg_recent_mood', 5) <= 2.5:
            suicidal_score += 0.4
        if row['habit_completion_rate'] <= 0.2:
            suicidal_score += 0.3
        
        scores['suicidal_ideation'] = min(1.0, suicidal_score)
        
        # Depression score
        depression_score = 0
        depression_score += (10 - row['mood_score']) / 10 * self.weights['mood_weight']
        depression_score += (1 - row['habit_completion_rate']) * self.weights['habits_weight']
        depression_score += (row['depression_keyword_count'] / 5) * self.weights['text_weight']
        depression_score += (row['dass_depression'] / 10) * self.weights['dass_weight']
        if row.get('mood_trend', 0) < -0.5:
            depression_score += 0.2 * self.weights['trend_weight']
        
        scores['depression'] = min(1.0, depression_score)
        
        # Anxiety score
        anxiety_score = 0
        anxiety_score += (10 - row['mood_score']) / 15 * self.weights['mood_weight']
        anxiety_score += (1 - row['habit_completion_rate']) * 0.5 * self.weights['habits_weight']
        anxiety_score += (row['anxiety_keyword_count'] / 4) * self.weights['text_weight']
        anxiety_score += (row['dass_anxiety'] / 10) * self.weights['dass_weight']
        anxiety_score += (row.get('mood_volatility', 0) / 3) * self.weights['trend_weight']
        
        scores['anxiety'] = min(1.0, anxiety_score)
        
        # Stress score
        stress_score = 0
        stress_score += (10 - row['mood_score']) / 15 * self.weights['mood_weight']
        stress_score += (1 - row['habit_completion_rate']) * 0.7 * self.weights['habits_weight']
        stress_score += (row['stress_keyword_count'] / 3) * self.weights['text_weight']
        stress_score += (row['dass_stress'] / 10) * self.weights['dass_weight']
        
        scores['stress'] = min(1.0, stress_score)
        
        # Overall wellness score
        wellness_score = (row['mood_score'] / 10) * 0.4
        wellness_score += row['habit_completion_rate'] * 0.3
        wellness_score += row['sentiment_ratio'] * 0.2
        if row.get('mood_trend', 0) > 0:
            wellness_score += 0.1
        
        scores['wellness'] = min(1.0, wellness_score)
        
        return scores
    
    def classify_condition(self, row):
        """Classify the primary mental health condition"""
        scores = self.calculate_risk_scores(row)
        
        # Priority-based classification
        if scores['suicidal_ideation'] >= self.thresholds['suicidal_ideation']:
            return 'suicidal_ideation', scores['suicidal_ideation']
        elif scores['depression'] >= self.thresholds['severe_depression']:
            return 'severe_depression', scores['depression']
        elif scores['anxiety'] >= self.thresholds['severe_anxiety']:
            return 'severe_anxiety', scores['anxiety']
        elif scores['stress'] >= self.thresholds['severe_stress']:
            return 'severe_stress', scores['stress']
        elif scores['depression'] >= self.thresholds['moderate_depression']:
            return 'moderate_depression', scores['depression']
        elif scores['anxiety'] >= self.thresholds['moderate_anxiety']:
            return 'moderate_anxiety', scores['anxiety']
        elif scores['stress'] >= self.thresholds['moderate_stress']:
            return 'moderate_stress', scores['stress']
        elif scores['wellness'] >= self.thresholds['healthy']:
            return 'healthy', scores['wellness']
        else:
            return 'mild_concern', max(scores['depression'], scores['anxiety'], scores['stress'])

# Step 4: Model Implementation with Assessment Function
class MentalHealthAssessment:
    def __init__(self, setup, classifier):
        self.setup = setup
        self.classifier = classifier
        self.engineer = FeatureEngineer(setup)
        
    def assess_mental_health(self, mood_score: float, habits_completed: float, 
                           text_input: str, recent_trends: Dict = None, 
                           dass_scores: Dict = None) -> Dict:
        """
        Main assessment function that takes user input and returns condition + resources
        """
        
        # Set defaults
        if recent_trends is None:
            recent_trends = {
                'mood_trend': 0,
                'habit_trend': 0,
                'avg_recent_mood': mood_score,
                'avg_recent_habits': habits_completed,
                'mood_volatility': 0
            }
            
        if dass_scores is None:
            dass_scores = {'depression': 5, 'anxiety': 5, 'stress': 5}
            
        # Create a mock row for processing
        mock_row = {
            'mood_score': mood_score,
            'habit_completion_rate': habits_completed,
            'text_entry': text_input,
            'dass_depression': dass_scores.get('depression', 5),
            'dass_anxiety': dass_scores.get('anxiety', 5),
            'dass_stress': dass_scores.get('stress', 5),
            **recent_trends
        }
        
        # Add text features
        text_features = self.engineer.extract_text_features(text_input)
        mock_row.update(text_features)
        
        # Get classification
        condition, confidence = self.classifier.classify_condition(mock_row)
        
        # Get risk scores for explanation
        risk_scores = self.classifier.calculate_risk_scores(mock_row)
        
        # Generate explanation
        explanation = self._generate_explanation(mock_row, risk_scores)
        
        # Get appropriate resources
        resources = self._get_resources(condition, confidence)
        
        return {
            'condition': condition,
            'confidence_score': round(confidence, 3),
            'risk_scores': {k: round(v, 3) for k, v in risk_scores.items()},
            'explanation': explanation,
            'resources': resources,
            'requires_confirmation': confidence > 0.7,
            'immediate_intervention_needed': condition == 'suicidal_ideation'
        }
    
    def _generate_explanation(self, row: Dict, risk_scores: Dict) -> str:
        """Generate human-readable explanation for the assessment"""
        explanations = []
        
        # Mood factors
        if row['mood_score'] <= 3:
            explanations.append(f"Low mood score ({row['mood_score']}/10) indicates distress")
        elif row['mood_score'] >= 8:
            explanations.append(f"Good mood score ({row['mood_score']}/10) is positive")
            
        # Habit factors
        if row['habit_completion_rate'] <= 0.3:
            explanations.append(f"Low habit completion ({row['habit_completion_rate']:.1%}) suggests difficulty with self-care")
        elif row['habit_completion_rate'] >= 0.7:
            explanations.append(f"Good habit completion ({row['habit_completion_rate']:.1%}) shows healthy routines")
            
        # Text analysis
        if row['suicidal_keyword_count'] > 0:
            explanations.append("Concerning language detected indicating possible suicidal thoughts")
        elif row['depression_keyword_count'] > 2:
            explanations.append("Language patterns suggest depressive symptoms")
        elif row['anxiety_keyword_count'] > 2:
            explanations.append("Language patterns suggest anxiety symptoms")
        elif row['positive_keyword_count'] > 0:
            explanations.append("Positive language detected")
            
        return ". ".join(explanations) if explanations else "Assessment based on overall patterns."
    
    def _get_resources(self, condition: str, confidence: float) -> Dict:
        """Get appropriate resources based on condition"""
        resource_map = {
            'suicidal_ideation': {
                'priority': 'CRITICAL',
                'immediate_action': [
                    "🚨 If you're in immediate danger, call 911",
                    "📞 National Suicide Prevention Lifeline: 988",
                    "💬 Crisis Text Line: Text HOME to 741741"
                ],
                'supportive_actions': [
                    "Reach out to a trusted friend or family member immediately",
                    "Remove means of self-harm from your environment",
                    "Create a safety plan with specific coping strategies"
                ],
                'self_care': [
                    "Stay with someone you trust",
                    "Focus on your breathing - try box breathing"
                ]
            },
            'severe_depression': {
                'priority': 'HIGH',
                'immediate_action': [
                    "📞 Consider contacting a mental health professional",
                    "👥 Reach out to trusted friends or family"
                ],
                'supportive_actions': [
                    "Schedule pleasant activities, even small ones",
                    "Maintain a daily routine",
                    "Practice behavioral activation"
                ],
                'self_care': [
                    "Prioritize sleep hygiene (7-9 hours)",
                    "Take a 10-minute walk outside",
                    "Practice gratitude - write down 3 things you're grateful for"
                ]
            },
            'severe_anxiety': {
                'priority': 'HIGH',
                'immediate_action': [
                    "Use grounding techniques immediately",
                    "Practice deep breathing exercises"
                ],
                'supportive_actions': [
                    "Challenge catastrophic thoughts",
                    "Use progressive muscle relaxation",
                    "Limit caffeine and stimulants"
                ],
                'self_care': [
                    "Box breathing: 4 counts in, 4 hold, 4 out, 4 hold",
                    "5-4-3-2-1 grounding: 5 things you see, 4 hear, 3 feel, 2 smell, 1 taste"
                ]
            },
            'severe_stress': {
                'priority': 'HIGH',
                'immediate_action': [
                    "Take immediate steps to reduce stressors",
                    "Practice stress management techniques"
                ],
                'supportive_actions': [
                    "Prioritize tasks and delegate when possible",
                    "Set boundaries with work and commitments"
                ],
                'self_care': [
                    "Take regular breaks throughout the day",
                    "Practice deep breathing or meditation"
                ]
            },
            'healthy': {
                'priority': 'LOW',
                'immediate_action': [
                    "Continue your current positive practices"
                ],
                'supportive_actions': [
                    "Maintain your healthy habits",
                    "Continue social connections"
                ],
                'self_care': [
                    "Keep up your mindfulness practices",
                    "Maintain regular exercise"
                ]
            }
        }
        
        # Default for conditions not explicitly mapped
        default_resources = {
            'priority': 'MEDIUM',
            'immediate_action': ['Monitor your symptoms closely'],
            'supportive_actions': ['Consider speaking with a counselor'],
            'self_care': ['Practice daily mindfulness', 'Maintain regular sleep schedule']
        }
        
        return resource_map.get(condition, default_resources)

# Step 5: Safety Validator
class SafetyValidator:
    def __init__(self):
        self.high_risk_conditions = ['suicidal_ideation', 'severe_depression', 'severe_anxiety', 'severe_stress']
        
    def validate_assessment(self, assessment_result: Dict) -> Dict:
        """Validate assessment and add safety measures"""
        
        validation_result = {
            'is_valid': True,
            'safety_flags': [],
            'confirmation_needed': False,
            'immediate_intervention': False,
            'follow_up_required': False
        }
        
        condition = assessment_result['condition']
        confidence = assessment_result['confidence_score']
        
        # Check for immediate intervention needs
        if condition == 'suicidal_ideation':
            validation_result['immediate_intervention'] = True
            validation_result['safety_flags'].append('CRITICAL: Suicidal ideation detected')
            
        # Check for high-risk conditions
        if condition in self.high_risk_conditions:
            validation_result['safety_flags'].append(f'High-risk condition: {condition}')
            validation_result['follow_up_required'] = True
            
        # Check confidence threshold
        if confidence >= 0.7:
            validation_result['confirmation_needed'] = True
            
        return validation_result

# Step 6: Integration System
class MentalHealthIntegration:
    def __init__(self):
        self.setup = MentalHealthDataSetup()
        self.classifier = MentalHealthClassifier(self.setup)
        self.assessment = MentalHealthAssessment(self.setup, self.classifier)
        self.validator = SafetyValidator()
        
    def process_daily_checkin(self, user_id: str, checkin_data: Dict) -> Dict:
        """Process a complete daily check-in"""
        
        # Run assessment
        assessment_result = self.assessment.assess_mental_health(
            mood_score=checkin_data['mood_score'],
            habits_completed=checkin_data['habit_completion_rate'],
            text_input=checkin_data.get('text_entry', ''),
            recent_trends=checkin_data.get('recent_trends'),
            dass_scores=checkin_data.get('dass_scores')
        )
        
        # Validate results
        validation = self.validator.validate_assessment(assessment_result)
        
        return {
            'user_id': user_id,
            'assessment': assessment_result,
            'validation': validation,
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }

# DEMO FUNCTION 
def run_demo():
    """Run a demonstration of the complete system"""
    
    # Initialize system
    integration_system = MentalHealthIntegration()
    
    print("🧠 Mental Health Assessment System Demo")
    print("=" * 50)
    
    # Test case 1: User with concerning symptoms
    print("\n📋 Test Case 1: User with depressive symptoms")
    checkin_data_1 = {
        'mood_score': 2,
        'habit_completion_rate': 0.2,
        'text_entry': 'Feeling really hopeless today, everything seems meaningless and I have no energy',
        'dass_scores': {'depression': 8, 'anxiety': 6, 'stress': 5}
    }
    
    result_1 = integration_system.process_daily_checkin('user_001', checkin_data_1)
    print(f"Assessment: {result_1['assessment']['condition']}")
    print(f"Confidence: {result_1['assessment']['confidence_score']}")
    print(f"Explanation: {result_1['assessment']['explanation']}")
    print(f"Safety flags: {result_1['validation']['safety_flags']}")
    print(f"Immediate actions: {len(result_1['assessment']['resources']['immediate_action'])}")
    
    # Test case 2: User with anxiety
    print("\n📋 Test Case 2: User with anxiety symptoms")
    checkin_data_2 = {
        'mood_score': 4,
        'habit_completion_rate': 0.6,
        'text_entry': 'Really anxious about everything, heart racing and cant stop worrying about work',
        'dass_scores': {'depression': 3, 'anxiety': 9, 'stress': 7}
    }
    
    result_2 = integration_system.process_daily_checkin('user_002', checkin_data_2)
    print(f"Assessment: {result_2['assessment']['condition']}")
    print(f"Confidence: {result_2['assessment']['confidence_score']}")
    print(f"Priority: {result_2['assessment']['resources']['priority']}")
    
    # Test case 3: Healthy user
    print("\n📋 Test Case 3: Healthy user")
    checkin_data_3 = {
        'mood_score': 8,
        'habit_completion_rate': 0.9,
        'text_entry': 'Great day today! Feeling energetic and accomplished after my morning walk',
        'dass_scores': {'depression': 2, 'anxiety': 2, 'stress': 3}
    }
    
    result_3 = integration_system.process_daily_checkin('user_003', checkin_data_3)
    print(f"Assessment: {result_3['assessment']['condition']}")
    print(f"Confidence: {result_3['assessment']['confidence_score']}")
    print(f"Risk scores: {result_3['assessment']['risk_scores']}")
    
    # Test case 4: Crisis situation
    print("\n📋 Test Case 4: Crisis situation")
    checkin_data_4 = {
        'mood_score': 1,
        'habit_completion_rate': 0.1,
        'text_entry': 'I cant go on anymore, thinking about ending it all, no point in living',
        'dass_scores': {'depression': 10, 'anxiety': 8, 'stress': 9}
    }
    
    result_4 = integration_system.process_daily_checkin('user_004', checkin_data_4)
    print(f"Assessment: {result_4['assessment']['condition']}")
    print(f"Confidence: {result_4['assessment']['confidence_score']}")
    print(f"Immediate intervention needed: {result_4['assessment']['immediate_intervention_needed']}")
    print(f"Crisis resources: {result_4['assessment']['resources']['immediate_action']}")
    
    print("\n✅ Demo completed successfully!")
    return "All systems working correctly!"

# Quick test function for single assessments
def quick_test(mood, habits, text, depression=5, anxiety=5, stress=5):
    """Quick test function for individual assessments"""
    integration_system = MentalHealthIntegration()
    
    checkin_data = {
        'mood_score': mood,
        'habit_completion_rate': habits,
        'text_entry': text,
        'dass_scores': {'depression': depression, 'anxiety': anxiety, 'stress': stress}
    }
    
    result = integration_system.process_daily_checkin('test_user', checkin_data)
    
    print(f"Condition: {result['assessment']['condition']}")
    print(f"Confidence: {result['assessment']['confidence_score']}")
    print(f"Explanation: {result['assessment']['explanation']}")
    print(f"Resources: {result['assessment']['resources']['priority']} priority")
    
    return result

# Example usage:
# To run the full demo: run_demo()
# To test quickly: quick_test(3, 0.4, "feeling overwhelmed and stressed", depression=7)

In [2]:
run_demo()

🧠 Mental Health Assessment System Demo

📋 Test Case 1: User with depressive symptoms
Assessment: suicidal_ideation
Confidence: 1.0
Explanation: Low mood score (2/10) indicates distress. Low habit completion (20.0%) suggests difficulty with self-care
Safety flags: ['CRITICAL: Suicidal ideation detected', 'High-risk condition: suicidal_ideation']
Immediate actions: 3

📋 Test Case 2: User with anxiety symptoms
Assessment: healthy
Confidence: 0.34
Priority: LOW

📋 Test Case 3: Healthy user
Assessment: healthy
Confidence: 0.79
Risk scores: {'suicidal_ideation': 0, 'depression': 0.105, 'anxiety': 0.081, 'stress': 0.104, 'wellness': 0.79}

📋 Test Case 4: Crisis situation
Assessment: suicidal_ideation
Confidence: 1.0
Immediate intervention needed: True
Crisis resources: ["🚨 If you're in immediate danger, call 911", '📞 National Suicide Prevention Lifeline: 988', '💬 Crisis Text Line: Text HOME to 741741']

✅ Demo completed successfully!


'All systems working correctly!'

In [4]:
# Test a stressed user
quick_test(mood=4, habits=0.5, text="really overwhelmed with work", stress=8)

# Test a happy user  
quick_test(mood=9, habits=0.9, text="amazing day, feeling great!")

Condition: healthy
Confidence: 0.31
Explanation: Assessment based on overall patterns.
Resources: LOW priority
Condition: healthy
Confidence: 0.83
Explanation: Good mood score (9/10) is positive. Good habit completion (90.0%) shows healthy routines. Positive language detected
Resources: LOW priority


{'user_id': 'test_user',
 'assessment': {'condition': 'healthy',
  'confidence_score': 0.83,
  'risk_scores': {'suicidal_ideation': 0,
   'depression': 0.14,
   'anxiety': 0.124,
   'stress': 0.127,
   'wellness': 0.83},
  'explanation': 'Good mood score (9/10) is positive. Good habit completion (90.0%) shows healthy routines. Positive language detected',
  'resources': {'priority': 'LOW',
   'immediate_action': ['Continue your current positive practices'],
   'supportive_actions': ['Maintain your healthy habits',
    'Continue social connections'],
   'self_care': ['Keep up your mindfulness practices',
    'Maintain regular exercise']},
  'requires_confirmation': True,
  'immediate_intervention_needed': False},
 'validation': {'is_valid': True,
  'safety_flags': [],
  'confirmation_needed': True,
  'immediate_intervention': False,
  'follow_up_required': False},
 'timestamp': '2025-07-19 15:19:54'}